In [1]:
import os
import argparse
import logging
import time
import numpy as np
import numpy.random as npr
import matplotlib
import pandas as pd
matplotlib.use('agg')
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchdiffeq import odeint_adjoint as odeint
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
list(data.columns.values)

['index',
 'L1A_rot',
 'R1A_rot',
 'L2A_rot',
 'R2A_rot',
 'L3A_rot',
 'R3A_rot',
 'L1B_rot',
 'R1B_rot',
 'L2B_rot',
 'R2B_rot',
 'L3B_rot',
 'R3B_rot',
 'L1C_rot',
 'R1C_rot',
 'L2C_rot',
 'R2C_rot',
 'L3C_rot',
 'R3C_rot',
 'L1A_flex',
 'R1A_flex',
 'L2A_flex',
 'R2A_flex',
 'L3A_flex',
 'R3A_flex',
 'L1B_flex',
 'R1B_flex',
 'L2B_flex',
 'R2B_flex',
 'L3B_flex',
 'R3B_flex',
 'L1C_flex',
 'R1C_flex',
 'L2C_flex',
 'R2C_flex',
 'L3C_flex',
 'R3C_flex',
 'L1D_flex',
 'R1D_flex',
 'L2D_flex',
 'R2D_flex',
 'L3D_flex',
 'R3D_flex',
 'L1A_abduct',
 'R1A_abduct',
 'L2A_abduct',
 'R2A_abduct',
 'L3A_abduct',
 'R3A_abduct',
 'L1E_x',
 'R1E_x',
 'L2E_x',
 'R2E_x',
 'L3E_x',
 'R3E_x',
 'L1E_y',
 'R1E_y',
 'L2E_y',
 'R2E_y',
 'L3E_y',
 'R3E_y',
 'L1E_z',
 'R1E_z',
 'L2E_z',
 'R2E_z',
 'L3E_z',
 'R3E_z',
 'L1_phase',
 'R1_phase',
 'L2_phase',
 'R2_phase',
 'L3_phase',
 'R3_phase',
 'fnum',
 'fullfile',
 'walking_bout_number',
 'stimlen',
 'flyid',
 'date',
 'fictrac_delta_rot_lab_x_mms',
 'fic

In [2]:
data = pd.read_parquet('FeCO_Opto_Fly_Data/claw_extension_silencing_walking.pq', engine='pyarrow')
list(data.columns.values)[1:43]

['L1A_rot',
 'R1A_rot',
 'L2A_rot',
 'R2A_rot',
 'L3A_rot',
 'R3A_rot',
 'L1B_rot',
 'R1B_rot',
 'L2B_rot',
 'R2B_rot',
 'L3B_rot',
 'R3B_rot',
 'L1C_rot',
 'R1C_rot',
 'L2C_rot',
 'R2C_rot',
 'L3C_rot',
 'R3C_rot',
 'L1A_flex',
 'R1A_flex',
 'L2A_flex',
 'R2A_flex',
 'L3A_flex',
 'R3A_flex',
 'L1B_flex',
 'R1B_flex',
 'L2B_flex',
 'R2B_flex',
 'L3B_flex',
 'R3B_flex',
 'L1C_flex',
 'R1C_flex',
 'L2C_flex',
 'R2C_flex',
 'L3C_flex',
 'R3C_flex',
 'L1D_flex',
 'R1D_flex',
 'L2D_flex',
 'R2D_flex',
 'L3D_flex',
 'R3D_flex']

## Selecting Timepoints

In [136]:
Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

#Copy to create new column 
Lookfor['AdditionValue'] = Lookfor['L1_step_num']

#Replace the copied value with addition of all the step num to create a data with all 6 actual values
for i in range(data.shape[0]):
    Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

#Create Angle Extraction Data
AngleExtraction = data.loc[:, list(data.columns.values)[1:43]]
AngleExtraction[Lookfor['AdditionValue'].notnull()]

Angles = pd.DataFrame(AngleExtraction[Lookfor['AdditionValue'].notnull()]).to_numpy()

In [138]:
AngleExtraction[Lookfor['AdditionValue'].notnull()]

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,L2C_flex,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex
34,-136.622753,-49.465247,-180.018402,-339.018566,-223.901142,-331.670494,-205.866503,-142.616145,-235.746033,-145.892156,...,79.806851,66.147856,84.959673,48.765092,112.589930,134.428387,136.676913,127.349747,151.352484,140.520550
35,-137.020855,-49.456088,-180.822699,-338.935534,-223.992516,-331.528113,-204.999340,-141.750057,-235.265398,-147.716793,...,78.698671,65.787919,86.151598,47.781799,113.882660,135.273635,136.206726,127.435787,151.928339,143.302514
36,-137.586970,-49.309828,-181.402827,-338.908121,-223.893708,-330.736128,-204.724992,-141.585640,-234.873988,-150.088523,...,77.905526,65.181137,87.250088,47.886942,115.179824,135.655545,135.487835,125.784229,152.252300,145.879424
37,-138.250386,-49.310936,-182.052444,-338.993105,-223.752977,-329.848402,-205.515718,-142.355480,-234.734126,-152.089088,...,76.997922,64.904672,88.165471,48.833256,116.739590,135.779402,134.932769,123.921621,152.295781,148.083771
38,-138.712798,-49.645277,-182.612135,-339.248869,-223.743272,-329.465327,-206.294133,-144.183631,-234.529365,-153.595051,...,76.339331,64.888708,89.083482,50.388885,118.332505,135.802125,134.495202,122.523329,152.637726,150.068135
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150840,-126.090215,-51.068367,-206.876310,-337.539328,-219.019580,-297.114652,-188.845176,-146.230401,-228.041078,-147.663819,...,59.531676,64.863949,49.265201,114.019696,129.628511,138.888468,129.055422,122.384011,134.849997,148.394601
150841,-126.428163,-58.791746,-207.261264,-336.538285,-219.380271,-297.393032,-188.386483,-150.649469,-236.605438,-149.527901,...,63.229155,66.895306,53.390224,119.180478,128.090674,137.527118,127.325851,123.365332,136.247370,154.301536
150842,-126.848265,-64.470298,-205.162254,-335.725645,-219.798005,-297.338422,-188.170147,-152.065132,-243.526806,-152.609619,...,67.770029,69.233741,56.384976,121.110667,128.017554,136.388537,125.927753,125.503125,136.349284,154.800280
150843,-127.522172,-67.827306,-200.837516,-334.716818,-220.539713,-297.443222,-188.056425,-152.513390,-244.905187,-155.143287,...,72.038066,70.790393,57.719273,119.727827,128.540556,135.407665,126.168523,126.658350,134.240530,153.135705


### Add Stimulation On/Off

In [64]:
AngleExtraction_stim.shape

(150875, 43)

In [3]:
X = list(data.columns.values)[1:43]
X.append('stim_on_off')

Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

#Copy to create new column 
Lookfor['AdditionValue'] = Lookfor['L1_step_num']

#Replace the copied value with addition of all the step num to create a data with all 6 actual values
for i in range(data.shape[0]):
    Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

#Create Angle Extraction Data
AngleExtraction_stim = data.loc[:, X]
Angles_stim = AngleExtraction_stim[Lookfor['AdditionValue'].notnull()]
#Angles_stim = pd.DataFrame(AngleExtraction_stim[Lookfor['AdditionValue'].notnull()]).to_numpy()
AngleExtraction_stim
#Angles_speed = pd.DataFrame(AngleExtraction_speed[Lookfor['AdditionValue'].notnull()]).to_numpy()

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,stim_on_off
0,-102.858882,-29.469508,-180.335542,-354.592568,-213.328418,-322.448615,-204.870374,-145.346751,-226.498937,-139.636660,...,40.987598,123.551710,63.243759,119.190786,132.347504,93.917586,97.062117,132.697730,126.796733,0
1,-109.490454,-25.719854,-193.713466,-355.174401,-220.797851,-323.538396,-201.553851,-141.651556,-227.867959,-139.480613,...,52.640571,102.853947,66.846755,126.698496,132.718283,114.535397,107.932047,138.129997,134.019628,0
2,-114.093455,-22.893619,-202.478459,-355.348699,-224.355667,-324.819672,-199.750314,-136.788119,-230.619271,-139.230353,...,62.729834,89.129560,69.753847,131.419112,131.066191,128.474676,117.897478,141.165173,139.788408,0
3,-117.152467,-20.663825,-205.570557,-354.946374,-224.437669,-325.410122,-199.647134,-130.588860,-231.202744,-138.650631,...,69.686030,79.648613,71.539722,134.004391,129.676187,134.410372,125.409118,142.120348,142.879237,0
4,-117.541982,-379.497081,-205.771496,-354.047472,-222.810143,-325.354614,-198.994319,-124.439346,-229.640563,-137.836543,...,73.218942,71.605292,73.069960,133.914177,129.464121,135.959220,129.772241,141.330552,143.841024,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150870,-119.709842,-44.201272,-206.704276,-338.129201,-219.238039,-306.556161,-190.797949,-140.577298,-223.150317,-166.569028,...,55.677873,54.132463,110.946432,127.302722,135.832766,126.239539,147.142658,133.106981,145.699739,0
150871,-119.529686,-52.265103,-207.565410,-336.874803,-219.547621,-307.938608,-188.850420,-146.976006,-232.432807,-170.921665,...,56.165443,57.302549,100.678528,125.541133,136.471947,123.626023,146.046121,133.936381,141.601288,0
150872,-119.041697,-58.891072,-206.272361,-335.732047,-220.414221,-310.064064,-187.553544,-153.668236,-240.194829,-175.069823,...,57.723226,60.839799,89.872206,124.817211,135.080483,123.275760,142.905531,136.551740,140.899211,0
150873,-119.940210,-63.272534,-203.006285,-334.954586,-221.317946,-313.803022,-187.990634,-157.334835,-242.009786,-176.229492,...,60.739722,64.571453,79.112131,124.533424,134.955187,124.306163,139.497098,138.554239,141.330479,0


In [86]:
AngleExtraction_stim[AngleExtraction_stim['stim_on_off']==1]

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,stim_on_off
150,-127.653654,-63.473478,-171.275338,-339.754913,-222.877093,-323.613952,-196.188745,-143.834519,-237.988574,-138.832486,...,53.144809,125.324690,94.962014,125.307945,131.432627,133.774277,115.089240,169.018192,159.716059,1
151,-127.422763,-62.580652,-170.617773,-339.724648,-222.351375,-323.428942,-196.382788,-142.872546,-236.622844,-138.915983,...,53.913034,124.422890,93.962350,126.507807,131.577512,133.748465,115.253327,169.159767,159.356252,1
152,-127.462980,-61.862441,-170.711180,-339.539255,-222.142244,-323.333908,-197.349043,-142.326685,-235.809536,-138.654320,...,54.783963,123.757074,90.970659,127.003626,131.593374,134.316385,116.024039,168.999153,158.989582,1
153,-127.754670,-61.691917,-171.381008,-339.265292,-222.227450,-323.375429,-198.563014,-142.719694,-235.607465,-138.519846,...,55.592497,123.433294,85.948396,127.247944,131.708064,134.941274,117.207167,169.024528,158.264597,1
154,-127.946151,-61.393722,-172.479526,-339.158268,-222.153085,-323.696671,-199.517071,-142.953835,-235.732212,-139.158731,...,56.479829,123.519870,79.184886,127.790839,131.638858,135.483346,118.428880,169.403888,156.726120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150743,-135.550774,-52.496808,-196.635425,-330.965078,-219.894739,-316.288702,-198.804116,-163.547579,-223.078841,-131.823730,...,71.374832,75.317601,64.282039,138.447502,120.308769,124.723792,122.068498,138.067163,118.025765,1
150744,-135.680578,-51.092167,-198.020654,-332.258574,-218.724966,-315.063352,-197.967040,-164.978549,-221.241465,-127.618817,...,68.641746,64.923217,70.286469,138.805827,119.022627,126.498964,123.658135,134.300703,120.578783,1
150745,-135.804016,-50.170354,-198.778940,-334.877138,-217.547600,-314.068592,-197.152622,-168.845950,-220.057185,-129.459783,...,64.412104,58.652110,75.837337,138.498722,118.419216,127.501074,122.994048,134.576297,123.513546,1
150746,-134.952401,-48.369047,-199.824818,-338.199819,-216.579550,-313.327832,-195.382668,-171.359636,-220.190742,-133.321553,...,60.226188,53.080254,81.373923,138.630440,123.182347,128.242699,119.693406,133.820404,126.145596,1


In [95]:
Angles_stim[Angles_stim['stim_on_off']==1][350:400]

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,stim_on_off
1654,-133.682988,-370.507194,-207.215949,-327.426700,-213.756145,-304.759306,-208.474386,-148.616989,-204.691120,-144.877447,...,48.941604,41.946862,112.913695,136.615364,125.453409,123.112495,120.405646,132.587991,159.698531,1
1655,-133.832522,-369.421401,-207.718397,-327.123422,-213.996956,-304.668537,-209.847383,-139.083608,-205.501020,-146.879879,...,50.339294,43.210617,116.236672,132.940321,127.713536,124.606904,121.919790,133.298269,160.659056,1
1656,-134.065569,-373.408418,-208.710292,-326.887789,-214.212455,-304.694850,-211.119595,-133.783639,-207.435441,-148.866229,...,51.804109,44.378017,119.552660,129.260276,130.207374,126.279672,123.894763,134.019008,161.748214,1
1657,-134.316025,-22.216766,-210.035178,-326.661071,-214.376972,-304.601805,-213.238045,-134.074635,-211.530850,-150.861344,...,53.302047,45.435238,122.978574,126.887339,130.961293,127.854860,126.156303,134.889317,162.687141,1
1658,-134.737904,-32.601247,-211.401382,-326.450342,-214.570995,-304.149752,-216.184097,-136.872056,-218.415864,-152.838828,...,54.782319,46.441369,126.851968,126.244990,131.305948,128.386338,128.309357,135.848320,163.515747,1
1659,-135.310122,-41.249578,-212.349950,-326.298648,-215.040534,-303.225107,-218.315466,-139.656802,-228.101370,-154.518777,...,56.221377,47.246741,131.152023,126.224593,132.269578,126.637524,130.066969,136.824111,164.277077,1
1660,-136.013322,-47.017164,-212.793513,-326.168521,-215.745601,-301.885000,-219.131743,-141.407074,-238.846211,-155.478335,...,57.304787,47.838655,135.342375,126.693982,134.116794,123.758789,130.994919,137.806336,165.021915,1
1661,-136.772544,-50.511538,-213.136110,-326.102454,-216.509621,-300.221974,-218.821255,-142.197406,-249.894055,-155.862067,...,57.887133,48.231585,139.276879,127.464560,135.848895,121.505851,131.133559,138.702947,165.871734,1
1662,-137.351131,-52.577268,-213.338018,-326.043147,-217.109888,-298.405205,-218.232781,-142.456458,-259.534990,-156.054521,...,57.848689,48.531746,142.645092,127.770874,136.819813,121.589705,130.415743,139.203404,166.495315,1
1663,-137.850581,-53.722465,-212.208883,-325.934463,-217.494789,-296.761198,-217.937411,-142.615984,-266.381931,-156.455304,...,57.222472,48.694810,145.183768,126.464724,136.924633,123.424037,128.695142,139.262705,166.981797,1


### Tryingto add Speed

In [22]:
#'bout_avg_speed'
X = list(data.columns.values)[1:43]
X.append('bout_avg_speed')

In [23]:
Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

#Copy to create new column 
Lookfor['AdditionValue'] = Lookfor['L1_step_num']

#Replace the copied value with addition of all the step num to create a data with all 6 actual values
for i in range(data.shape[0]):
    Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

#Create Angle Extraction Data
AngleExtraction_speed = data.loc[:, X]
AngleExtraction_speed[Lookfor['AdditionValue'].notnull()]

Angles_speed = pd.DataFrame(AngleExtraction_speed[Lookfor['AdditionValue'].notnull()]).to_numpy()

In [63]:
Angles_speed.shape

NameError: name 'Angles_speed' is not defined

In [24]:
AngleExtraction_speed

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,bout_avg_speed
0,-102.858882,-29.469508,-180.335542,-354.592568,-213.328418,-322.448615,-204.870374,-145.346751,-226.498937,-139.636660,...,40.987598,123.551710,63.243759,119.190786,132.347504,93.917586,97.062117,132.697730,126.796733,5.226395
1,-109.490454,-25.719854,-193.713466,-355.174401,-220.797851,-323.538396,-201.553851,-141.651556,-227.867959,-139.480613,...,52.640571,102.853947,66.846755,126.698496,132.718283,114.535397,107.932047,138.129997,134.019628,5.226395
2,-114.093455,-22.893619,-202.478459,-355.348699,-224.355667,-324.819672,-199.750314,-136.788119,-230.619271,-139.230353,...,62.729834,89.129560,69.753847,131.419112,131.066191,128.474676,117.897478,141.165173,139.788408,5.226395
3,-117.152467,-20.663825,-205.570557,-354.946374,-224.437669,-325.410122,-199.647134,-130.588860,-231.202744,-138.650631,...,69.686030,79.648613,71.539722,134.004391,129.676187,134.410372,125.409118,142.120348,142.879237,5.226395
4,-117.541982,-379.497081,-205.771496,-354.047472,-222.810143,-325.354614,-198.994319,-124.439346,-229.640563,-137.836543,...,73.218942,71.605292,73.069960,133.914177,129.464121,135.959220,129.772241,141.330552,143.841024,5.226395
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150870,-119.709842,-44.201272,-206.704276,-338.129201,-219.238039,-306.556161,-190.797949,-140.577298,-223.150317,-166.569028,...,55.677873,54.132463,110.946432,127.302722,135.832766,126.239539,147.142658,133.106981,145.699739,13.645840
150871,-119.529686,-52.265103,-207.565410,-336.874803,-219.547621,-307.938608,-188.850420,-146.976006,-232.432807,-170.921665,...,56.165443,57.302549,100.678528,125.541133,136.471947,123.626023,146.046121,133.936381,141.601288,13.645840
150872,-119.041697,-58.891072,-206.272361,-335.732047,-220.414221,-310.064064,-187.553544,-153.668236,-240.194829,-175.069823,...,57.723226,60.839799,89.872206,124.817211,135.080483,123.275760,142.905531,136.551740,140.899211,13.645840
150873,-119.940210,-63.272534,-203.006285,-334.954586,-221.317946,-313.803022,-187.990634,-157.334835,-242.009786,-176.229492,...,60.739722,64.571453,79.112131,124.533424,134.955187,124.306163,139.497098,138.554239,141.330479,13.645840


## Lets now create a method to use it in multiple files in a single folder

In [183]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles'

if not os.path.exists(folder_path):
    # If it doesn't exist, create it
    os.makedirs(folder_path)

for i in range(len(dir_list)):
    uploadpath = path + '/' + dir_list[i]
    pathfile = folder_path + '/' + dir_list[i]
    pathfile_npy = pathfile.replace('pq', 'npy')
    
    #upload data
    data = pd.read_parquet(uploadpath, engine='pyarrow')
    
    #search for legs
    Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

    #Copy to create new column 
    Lookfor['AdditionValue'] = Lookfor['L1_step_num']

    #Replace the copied value with addition of all the step num to create a data with all 6 actual values
    for i in range(data.shape[0]):
        Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

    #Create Angle Extraction Data
    AngleExtraction = data.loc[:, list(data.columns.values)[1:43]]
    AngleExtraction[Lookfor['AdditionValue'].notnull()]

    Angles = pd.DataFrame(AngleExtraction[Lookfor['AdditionValue'].notnull()]).to_numpy()
    
    np.save(pathfile_npy, Angles)

In [6]:
AngleExtraction_stim[AngleExtraction_stim['stim_on_off']==1]

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,stim_on_off
150,-127.653654,-63.473478,-171.275338,-339.754913,-222.877093,-323.613952,-196.188745,-143.834519,-237.988574,-138.832486,...,53.144809,125.324690,94.962014,125.307945,131.432627,133.774277,115.089240,169.018192,159.716059,1
151,-127.422763,-62.580652,-170.617773,-339.724648,-222.351375,-323.428942,-196.382788,-142.872546,-236.622844,-138.915983,...,53.913034,124.422890,93.962350,126.507807,131.577512,133.748465,115.253327,169.159767,159.356252,1
152,-127.462980,-61.862441,-170.711180,-339.539255,-222.142244,-323.333908,-197.349043,-142.326685,-235.809536,-138.654320,...,54.783963,123.757074,90.970659,127.003626,131.593374,134.316385,116.024039,168.999153,158.989582,1
153,-127.754670,-61.691917,-171.381008,-339.265292,-222.227450,-323.375429,-198.563014,-142.719694,-235.607465,-138.519846,...,55.592497,123.433294,85.948396,127.247944,131.708064,134.941274,117.207167,169.024528,158.264597,1
154,-127.946151,-61.393722,-172.479526,-339.158268,-222.153085,-323.696671,-199.517071,-142.953835,-235.732212,-139.158731,...,56.479829,123.519870,79.184886,127.790839,131.638858,135.483346,118.428880,169.403888,156.726120,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150743,-135.550774,-52.496808,-196.635425,-330.965078,-219.894739,-316.288702,-198.804116,-163.547579,-223.078841,-131.823730,...,71.374832,75.317601,64.282039,138.447502,120.308769,124.723792,122.068498,138.067163,118.025765,1
150744,-135.680578,-51.092167,-198.020654,-332.258574,-218.724966,-315.063352,-197.967040,-164.978549,-221.241465,-127.618817,...,68.641746,64.923217,70.286469,138.805827,119.022627,126.498964,123.658135,134.300703,120.578783,1
150745,-135.804016,-50.170354,-198.778940,-334.877138,-217.547600,-314.068592,-197.152622,-168.845950,-220.057185,-129.459783,...,64.412104,58.652110,75.837337,138.498722,118.419216,127.501074,122.994048,134.576297,123.513546,1
150746,-134.952401,-48.369047,-199.824818,-338.199819,-216.579550,-313.327832,-195.382668,-171.359636,-220.190742,-133.321553,...,60.226188,53.080254,81.373923,138.630440,123.182347,128.242699,119.693406,133.820404,126.145596,1


In [16]:
pd.DataFrame(AngleExtraction[Lookfor['AdditionValue'].notnull()]).to_numpy()

(150875, 43)

### Stim added

In [ ]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'

X = list(data.columns.values)[1:43]
X.append('stim_on_off')

if not os.path.exists(folder_path):
    # If it doesn't exist, create it
    os.makedirs(folder_path)

for i in range(len(dir_list)):
    uploadpath = path + '/' + dir_list[i]
    pathfile = folder_path + '/' + dir_list[i]
    pathfile_npy = pathfile.replace('pq', 'npy')
    
    #upload data
    data = pd.read_parquet(uploadpath, engine='pyarrow')
    
    #search for legs
    Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

    #Copy to create new column 
    Lookfor['AdditionValue'] = Lookfor['L1_step_num']

    #Replace the copied value with addition of all the step num to create a data with all 6 actual values
    for i in range(data.shape[0]):
        Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

    #Create Angle Extraction Data
    AngleExtraction_stim = data.loc[:, X]
    #AngleExtraction_stim[Lookfor['AdditionValue'].notnull()]

    AngleExtraction_stim_n = AngleExtraction_stim[Lookfor['AdditionValue'].notnull()]
    AngleExtraction_stim_off = pd.DataFrame(AngleExtraction_stim_n[AngleExtraction_stim_n['stim_on_off'] == 0]).to_numpy()
    AngleExtraction_stim_on = pd.DataFrame(AngleExtraction_stim_n[AngleExtraction_stim_n['stim_on_off'] == 1]).to_numpy()
    
    stim_off_path = pathfile_npy.replace('.npy', '_off.npy')
    stim_on_path = pathfile_npy.replace('.npy', '_on.npy')
    
    np.save(stim_off_path, AngleExtraction_stim_off)
    np.save(stim_on_path, AngleExtraction_stim_on)

#### Speed added

In [27]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_speed'

if not os.path.exists(folder_path):
    # If it doesn't exist, create it
    os.makedirs(folder_path)

for i in range(len(dir_list)):
    uploadpath = path + '/' + dir_list[i]
    pathfile = folder_path + '/' + dir_list[i]
    pathfile_npy = pathfile.replace('pq', 'npy')
    
    #upload data
    data = pd.read_parquet(uploadpath, engine='pyarrow')
    
    #search for legs
    Lookfor = data.loc[:, ['L1_step_num', 'L2_step_num', 'L3_step_num','R1_step_num', 'R2_step_num', 'R3_step_num']]

    #Copy to create new column 
    Lookfor['AdditionValue'] = Lookfor['L1_step_num']

    #Replace the copied value with addition of all the step num to create a data with all 6 actual values
    for i in range(data.shape[0]):
        Lookfor['AdditionValue'][i] = data['L1_step_num'][i]+ data['L2_step_num'][i]+ data['L3_step_num'][i]+ data['R1_step_num'][i]+data['R2_step_num'][i]+data['R3_step_num'][i]

    #Create Angle Extraction Data
    AngleExtraction_speed = data.loc[:, X]
    AngleExtraction_speed[Lookfor['AdditionValue'].notnull()]

    Angles_speed = pd.DataFrame(AngleExtraction_speed[Lookfor['AdditionValue'].notnull()]).to_numpy()
    
    np.save(pathfile_npy, Angles_speed)

## Graph the data truncated to see how different it is from the original data

In [2]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles'
original_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
Truncatedfolder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/TruncatedComparisonPlots'

if not os.path.exists(Truncatedfolder_path):
    # If it doesn't exist, create it
    os.makedirs(Truncatedfolder_path)

for k in range(len(os.listdir(folder_path))):
    fig, axes = plt.subplots(nrows=7, ncols=6, figsize=(50, 30))
    axes = axes.flatten()
    K = np.arange(0, 42)
    for j, ax in enumerate(axes[K]):
        Y = np.load(folder_path + '/' + os.listdir(folder_path)[k])
        X = np.arange(0, Y.shape[0])

        YB = pd.read_parquet(original_folder_path + '/' + os.listdir(original_folder_path)[k], engine='pyarrow')
        Y1 = pd.DataFrame(YB.loc[:, list(YB.columns.values)[1:43]]).to_numpy()
        X1 = np.arange(0, Y1.shape[0])
        
        fig_name = list(YB.columns.values)[j+1]
        overall_name = Truncatedfolder_path + '/zoom' + os.listdir(folder_path)[k].replace('.npy', '.png')
        #save_fig_name = Truncatedfolder_path + '/' + "OriginalvsTruncated_" + fig_name + '.png'
        ax.plot(X, Y[:,j], color="red", label = "Truncated")
        ax.plot(X1, Y1[:,j], color="blue", label = "Original")
        ax.set_xlim(31000, 31500)
        ax.legend(loc="upper right")
        ax.set_title(fig_name)
        plt.savefig(overall_name)
        
        

In [51]:
len(list(YB.columns.values)[1:43])

42

## Stimulated vs Not Stimulated

In [115]:
YB = pd.read_parquet(original_folder_path + '/' + os.listdir(original_folder_path)[0], engine='pyarrow')

In [40]:
len(os.listdir(folder_path))

16

In [34]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'
original_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
Stim_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/StimulationComparisonPlots'

if not os.path.exists(Stim_folder_path):
    # If it doesn't exist, create it
    os.makedirs(Stim_folder_path)

for k in range(len(os.listdir(folder_path))):
    fig, axes = plt.subplots(nrows=7, ncols=6, figsize=(50, 30))
    axes = axes.flatten()
    K = np.arange(0, 42)
    for j, ax in enumerate(axes[K]):
        Y = np.load(folder_path + '/' + os.listdir(folder_path)[2*k])
        X = np.arange(0, Y.shape[0])

        Y1 = np.load(folder_path + '/' + os.listdir(folder_path)[2*k+1])
        X1 = np.arange(0, Y1.shape[0])
        
        fig_name = list(YB.columns.values)[j+1]
        overall_name = Stim_folder_path + '/zoom' + os.listdir(folder_path)[2*k].replace('off.npy', 'onoff.png')
        #save_fig_name = Truncatedfolder_path + '/' + "OriginalvsTruncated_" + fig_name + '.png'
        ax.plot(X, Y[:,j], color="red", label = "Normal")
        ax.plot(X1, Y1[:,j], color="blue", label = "Stimulated")
        ax.set_xlim(1000, 2000)
        ax.legend(loc="upper right")
        ax.set_title(fig_name)
        plt.savefig(overall_name)

IndexError: list index out of range

### Stimulated Changes Plot

In [83]:
YB = pd.read_parquet(original_folder_path + '/' + os.listdir(original_folder_path)[0], engine='pyarrow')

In [65]:
data = pd.DataFrame(Angles_stim).to_numpy()
#Angles_stim[115:145][0:42]

In [75]:
Angles_stim[150:189][0:42]

,L1A_rot,R1A_rot,L2A_rot,R2A_rot,L3A_rot,R3A_rot,L1B_rot,R1B_rot,L2B_rot,R2B_rot,...,R2C_flex,L3C_flex,R3C_flex,L1D_flex,R1D_flex,L2D_flex,R2D_flex,L3D_flex,R3D_flex,stim_on_off
185,-115.837354,-43.090261,-187.176172,-360.141546,-216.277200,-324.849739,-199.437758,-153.802941,-233.944226,-124.827747,...,81.892930,48.710765,61.901289,140.136656,142.462876,136.913431,122.273406,140.219634,139.080738,0
186,-116.094249,-43.343119,-187.928636,-360.697240,-216.087263,-324.897477,-199.115211,-152.805212,-233.339538,-125.758817,...,79.024935,47.725053,63.498619,139.199202,139.568682,137.179967,122.761364,139.905116,139.110945,0
187,-116.556248,-42.220158,-188.813024,-360.005306,-216.319572,-324.970569,-199.528653,-150.060419,-232.531676,-126.261630,...,76.940781,47.223922,65.022920,139.013672,138.234775,137.811832,123.800658,139.526853,139.199305,0
188,-116.617801,-39.546947,-189.596206,-358.765856,-216.844120,-324.986674,-199.749539,-145.717881,-232.035627,-126.557954,...,75.362166,47.079678,66.437458,139.345653,138.563414,138.624400,124.637212,139.430930,139.490856,0
189,-116.759755,-36.033444,-190.563463,-357.511874,-217.316167,-324.796164,-200.100549,-140.315268,-231.480342,-127.004404,...,74.017186,47.188872,67.721743,139.487745,137.556436,139.009113,124.856617,139.588158,140.103923,0
190,-117.237826,-31.256434,-191.531771,-356.530965,-217.441651,-324.249652,-200.593085,-133.244231,-230.647618,-127.916399,...,72.943083,47.912591,68.988833,138.692073,135.359985,138.591165,124.872146,139.917933,140.992694,0
191,-117.944252,-25.715186,-192.378087,-355.824213,-217.447223,-323.598338,-201.560517,-125.342305,-229.971873,-129.225060,...,72.093776,48.628965,70.460626,137.218542,133.824212,137.807290,125.210308,140.224669,141.978208,0
192,-118.781997,-20.683388,-193.203583,-355.202908,-217.503891,-322.993637,-202.660816,-118.311689,-229.468731,-130.634004,...,71.259647,49.040310,71.953652,136.033275,133.530531,137.089149,125.507150,140.403382,142.766576,0
193,-119.596704,-378.314986,-194.020245,-354.549530,-217.775738,-322.552590,-203.145849,-114.897406,-229.108236,-131.409886,...,70.418249,49.471773,73.519834,134.988668,134.400762,136.732840,125.365645,140.293729,143.491699,0
194,-120.461912,-379.398391,-194.662823,-353.907513,-218.235961,-322.293871,-202.901192,-115.656945,-228.706316,-131.446614,...,69.603647,50.239522,75.287335,134.666778,136.151755,137.088378,124.785107,140.192820,144.237939,0


In [67]:
folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'
os.listdir(folder_path)

['claw_extension_activation_walking_off.npy',
 'claw_extension_activation_walking_on.npy',
 'claw_extension_silencing_walking_off.npy',
 'claw_extension_silencing_walking_on.npy',
 'claw_flexion_activation_walking_off.npy',
 'claw_flexion_activation_walking_on.npy',
 'claw_flexion_silencing_walking_off.npy',
 'claw_flexion_silencing_walking_on.npy',
 'hook_extension_activation_walking_off.npy',
 'hook_extension_activation_walking_on.npy',
 'hook_extension_silencing_walking_off.npy',
 'hook_extension_silencing_walking_on.npy',
 'hook_flexion_activation_walking_off.npy',
 'hook_flexion_activation_walking_on.npy',
 'hook_flexion_silencing_walking_off.npy',
 'hook_flexion_silencing_walking_on.npy']

In [78]:
    Y1 = data[115:145,:42]
    X = np.arange(115, 145)

In [84]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'
original_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
Stim_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/StimulationChangesPlots'



if not os.path.exists(Stim_folder_path):
    # If it doesn't exist, create it
    os.makedirs(Stim_folder_path)


fig, axes = plt.subplots(nrows=7, ncols=6, figsize=(50, 30))
axes = axes.flatten()
K = np.arange(0, 42)
for j, ax in enumerate(axes[K]):
    Y = data[:189,:42]
    X = np.arange(0, 189)

    Y1 = data[115:145,:42]
    X1 = np.arange(115, 145)

    fig_name = list(YB.columns.values)[j+1]
    overall_name = Stim_folder_path + '/zoom' + '.png'
    #save_fig_name = Truncatedfolder_path + '/' + "OriginalvsTruncated_" + fig_name + '.png'
    ax.plot(X, Y[:,j], color="blue", label = "Normal")
    ax.plot(X1, Y1[:,j], color="red", label = "Stimulated")
    ax.set_xlim(0, 189)
    ax.legend(loc="upper right")
    ax.set_title(fig_name)
    plt.savefig(overall_name)

## Lets normalize the data

### Normalized with On/Off

In [112]:
def load_data_normalize_Stimulation(obs_dim, datafilepath):
    data = np.load(datafilepath)
    traj_tot = data.copy()
    traj_tot = traj_tot[:,0:15000,:]
    data = data[:, 0:15000, :]
    data = data.reshape(data.shape[0], 15000, obs_dim)
    noise_std = 0.2

    orig_trajs = np.zeros((data.shape[0],data.shape[1],data.shape[2]))
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            trajs = data[i,:,j]
            trajs_tot = traj_tot[i,:,j]
            orig_trajs[i,:,j] = (trajs - trajs_tot.mean()) / trajs_tot.std()
            
    #samp_trajs += npr.randn(*samp_trajs.shape) * noise_std #add noise

    return orig_trajs


def takenembedding(data, tau, k, obs_dim, device):
    data_TE = np.zeros((data.shape[0], data.shape[1]-tau*k, data.shape[2]), dtype = object)
    
    for i in range(data.shape[0]):
        for j in range(data.shape[2]):
            for t in range(data.shape[1]-tau*k):
                data_TE[i,t,j] = data[i, t:t+tau*k+1, j][::tau][::-1]
                
    data_TE = np.array(data_TE.tolist())
    data_TE = data_TE.reshape(data_TE.shape[0],data_TE.shape[1],obs_dim*(k+1))
    
    return data_TE
    

In [138]:
Datafolder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'

In [141]:
np.load(os.path.join(Datafolder_path, os.listdir(Datafolder_path)[1])).shape

(31543, 43)

In [142]:
Datafolder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles_stim'
DataFlies_Stim_Trim_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFlies_stim_trim'

Concatenated_Stim_Flies_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/Concatenated_Flies_Stim_Data'

if not os.path.exists(Concatenated_Stim_Flies_path):
    # If it doesn't exist, create it
    os.makedirs(Concatenated_Stim_Flies_path)
if not os.path.exists(DataFlies_Stim_Trim_path):
    # If it doesn't exist, create it
    os.makedirs(DataFlies_Stim_Trim_path)
    
Concatenated_file_name = os.path.join(Concatenated_Stim_Flies_path, 'Flies_Concatenated.npy')
if not os.path.exists(DataFlies_Stim_Trim_path):
    # If it doesn't exist, create it
    os.makedirs(DataFlies_Stim_Trim_path)

for i in range(len(os.listdir(Datafolder_path))):
    X = np.load(os.path.join(Datafolder_path, os.listdir(Datafolder_path)[i]))
    X = X[1000:16000,:]
    np.save(os.path.join(DataFlies_Stim_Trim_path, os.listdir(Datafolder_path)[i]), X)

for i in range(2):
    X = np.load(os.path.join(Datafolder_path, os.listdir(Datafolder_path)[i]))
    X = X[15000:30000,:]
    np.save(os.path.join(DataFlies_Stim_Trim_path, os.listdir(Datafolder_path)[i]), X)


all_arrays = []
for npfile in os.listdir(DataFlies_Stim_Trim_path):
    all_arrays.append(np.load(os.path.join(DataFlies_Stim_Trim_path, npfile)))
all_arrays = np.array(all_arrays)
all_arrays = all_arrays[:,:,:42]
np.save(Concatenated_file_name, all_arrays)
#Test = load_data_normalize_Stimulation(X.shape[1], Concatenated_file_name, X)
    
normalized_concatenated_Flies = load_data_normalize_Stimulation(all_arrays.shape[2], Concatenated_file_name)

Normalized_Concatenated_Flies_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/Normalized_Concatenated_Flies_Stim_Data'
if not os.path.exists(Normalized_Concatenated_Flies_path):
    # If it doesn't exist, create it
    os.makedirs(Normalized_Concatenated_Flies_path)
    
Norm_Concatenated_file_name = os.path.join(Normalized_Concatenated_Flies_path, 'Flies_Concatenated_Norm.npy')
np.save(Norm_Concatenated_file_name, normalized_concatenated_Flies)

In [4]:
Normalized_Concatenated_Flies_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/Normalized_Concatenated_Flies_Stim_Data'
Norm_Concatenated_file_name = os.path.join(Normalized_Concatenated_Flies_path, 'Flies_Concatenated_Norm.npy')
normalized_concatenated_Flies = np.load(Norm_Concatenated_file_name)

In [7]:
good_ind = [18,19, 24, 25, 28, 29, 30, 31, 34, 35, 40, 41]
data = normalized_concatenated_Flies[:,:,good_ind]

In [8]:
data = data[:,::2,:]

In [9]:
data.shape

(16, 7500, 12)

In [26]:
fig, axes = plt.subplots(nrows=3, ncols=4, figsize=(15, 9))
axes = axes.flatten()
K = np.arange(0, 12)
for j, ax in enumerate(axes[K]):
    Y = data[15,0:7500,:]
    X = np.arange(0, 7500)

    fig_name = list(YB.columns.values)[j+1]
    overall_name = Stim_folder_path + '/zoom15' + '.png'
    #save_fig_name = Truncatedfolder_path + '/' + "OriginalvsTruncated_" + fig_name + '.png'
    ax.plot(X, Y[:,j], color="blue", label = "Normal")

    ax.set_xlim(0, 7500)
    ax.legend(loc="upper right")
    ax.set_title(fig_name)
    plt.savefig(overall_name)

In [22]:
Stim_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/StimulationChangesPlots'
original_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
YB = pd.read_parquet(original_folder_path + '/' + os.listdir(original_folder_path)[0], engine='pyarrow')

In [18]:
Stim_folder_path + '/zoom13' + '.png'

'C:/Users/shiny/Documents/NeuralODE_Fly/StimulationChangesPlots/zoom13.png'

### Normalize with Speed

In [6]:
Normalized_Concatenated_Flies_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/Normalized_Concatenated_Flies_Data'
Norm_Concatenated_file_name = os.path.join(Normalized_Concatenated_Flies_path, 'Flies_Concatenated_Norm.npy')
fps300data = np.load(Norm_Concatenated_file_name)
fps150data = fps300data[:,::2,:]

In [7]:
good_ind = [18,19, 22,23, 24, 25, 28,29,30,31, 34, 35 ,36, 37, 40, 41]
fps150data = fps150data[:,:, good_ind]

In [21]:
YB = pd.read_parquet(original_folder_path + '/' + os.listdir(original_folder_path)[0], engine='pyarrow')

NameError: name 'original_folder_path' is not defined

In [25]:
list(YB.columns.values)[good_ind[17]+1]

IndexError: list index out of range

In [26]:
path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
dir_list = os.listdir(path)

folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/DataFiles'
original_folder_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/FeCO_Opto_Fly_Data'
ActSilfolder_norm_path = 'C:/Users/shiny/Documents/NeuralODE_Fly/ActSil_Norm_ComparisonPlots'

if not os.path.exists(ActSilfolder_norm_path):
    # If it doesn't exist, create it
    os.makedirs(ActSilfolder_norm_path)

for k in range(np.int32(len(os.listdir(folder_path))/2)):
    fig, axes = plt.subplots(nrows=4, ncols=4, figsize=(50, 30))
    axes = axes.flatten()
    K = np.arange(0, 16)
    for j, ax in enumerate(axes[K]):
        Y = fps150data[2*k,:,:]
        X = np.arange(0, Y.shape[0])

        Y1 = fps150data[2*k+1,:,:]
        X1 = np.arange(0, Y1.shape[0])
        
        fig_name = list(YB.columns.values)[good_ind[j]+1]
        overall_name = ActSilfolder_norm_path + '/zoom' + os.listdir(folder_path)[2*k].replace('activation_walking.npy', 'Comparison.png')
        #save_fig_name = Truncatedfolder_path + '/' + "OriginalvsTruncated_" + fig_name + '.png'
        ax.plot(X, Y[:,j], color="red", label = "Activated")
        ax.plot(X1, Y1[:,j], color="blue", label = "Silenced")
        ax.set_xlim(21000, 21500)
        ax.legend(loc="upper right")
        ax.set_title(fig_name)
        plt.savefig(overall_name)